# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f179032bbb0>
├── 'a' --> tensor([[ 1.6485,  1.3529,  0.3517],
│                   [-1.5229,  0.5561,  0.2962]])
└── 'x' --> <FastTreeValue 0x7f17923b68b0>
    └── 'c' --> tensor([[ 0.0233,  0.4830, -1.0290, -0.6119],
                        [-0.3239,  0.0862, -0.1516,  0.3273],
                        [-1.6574,  1.5067,  0.3015,  1.0849]])

In [4]:
t.a

tensor([[ 1.6485,  1.3529,  0.3517],
        [-1.5229,  0.5561,  0.2962]])

In [5]:
%timeit t.a

87.4 ns ± 2 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f179032bbb0>
├── 'a' --> tensor([[-0.8718, -1.2017,  0.0520],
│                   [ 1.0355, -0.3283, -0.6864]])
└── 'x' --> <FastTreeValue 0x7f17923b68b0>
    └── 'c' --> tensor([[ 0.0233,  0.4830, -1.0290, -0.6119],
                        [-0.3239,  0.0862, -0.1516,  0.3273],
                        [-1.6574,  1.5067,  0.3015,  1.0849]])

In [7]:
%timeit t.a = new_value

110 ns ± 1.3 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.6485,  1.3529,  0.3517],
               [-1.5229,  0.5561,  0.2962]]),
    x: Batch(
           c: tensor([[ 0.0233,  0.4830, -1.0290, -0.6119],
                      [-0.3239,  0.0862, -0.1516,  0.3273],
                      [-1.6574,  1.5067,  0.3015,  1.0849]]),
       ),
)

In [10]:
b.a

tensor([[ 1.6485,  1.3529,  0.3517],
        [-1.5229,  0.5561,  0.2962]])

In [11]:
%timeit b.a

72.9 ns ± 1.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0366,  0.7024,  0.4510],
               [-0.4438,  0.8842, -0.0581]]),
    x: Batch(
           c: tensor([[ 0.0233,  0.4830, -1.0290, -0.6119],
                      [-0.3239,  0.0862, -0.1516,  0.3273],
                      [-1.6574,  1.5067,  0.3015,  1.0849]]),
       ),
)

In [13]:
%timeit b.a = new_value

666 ns ± 9.92 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.07 µs ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.4 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

305 µs ± 6.42 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

294 µs ± 7.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f17647ba6a0>
├── 'a' --> tensor([[[ 1.6485,  1.3529,  0.3517],
│                    [-1.5229,  0.5561,  0.2962]],
│           
│                   [[ 1.6485,  1.3529,  0.3517],
│                    [-1.5229,  0.5561,  0.2962]],
│           
│                   [[ 1.6485,  1.3529,  0.3517],
│                    [-1.5229,  0.5561,  0.2962]],
│           
│                   [[ 1.6485,  1.3529,  0.3517],
│                    [-1.5229,  0.5561,  0.2962]],
│           
│                   [[ 1.6485,  1.3529,  0.3517],
│                    [-1.5229,  0.5561,  0.2962]],
│           
│                   [[ 1.6485,  1.3529,  0.3517],
│                    [-1.5229,  0.5561,  0.2962]],
│           
│                   [[ 1.6485,  1.3529,  0.3517],
│                    [-1.5229,  0.5561,  0.2962]],
│           
│                   [[ 1.6485,  1.3529,  0.3517],
│                    [-1.5229,  0.5561,  0.2962]]])
└── 'x' --> <FastTreeValue 0x7f16e3610a30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.4 µs ± 619 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f16e2ff5f40>
├── 'a' --> tensor([[ 1.6485,  1.3529,  0.3517],
│                   [-1.5229,  0.5561,  0.2962],
│                   [ 1.6485,  1.3529,  0.3517],
│                   [-1.5229,  0.5561,  0.2962],
│                   [ 1.6485,  1.3529,  0.3517],
│                   [-1.5229,  0.5561,  0.2962],
│                   [ 1.6485,  1.3529,  0.3517],
│                   [-1.5229,  0.5561,  0.2962],
│                   [ 1.6485,  1.3529,  0.3517],
│                   [-1.5229,  0.5561,  0.2962],
│                   [ 1.6485,  1.3529,  0.3517],
│                   [-1.5229,  0.5561,  0.2962],
│                   [ 1.6485,  1.3529,  0.3517],
│                   [-1.5229,  0.5561,  0.2962],
│                   [ 1.6485,  1.3529,  0.3517],
│                   [-1.5229,  0.5561,  0.2962]])
└── 'x' --> <FastTreeValue 0x7f17649eb9d0>
    └── 'c' --> tensor([[ 0.0233,  0.4830, -1.0290, -0.6119],
                        [-0.3239,  0.0862, -0.1516,  0.3273],
                 

In [23]:
%timeit t_cat(trees)

43.7 µs ± 827 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

83.5 µs ± 663 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.6485,  1.3529,  0.3517],
                [-1.5229,  0.5561,  0.2962]],
       
               [[ 1.6485,  1.3529,  0.3517],
                [-1.5229,  0.5561,  0.2962]],
       
               [[ 1.6485,  1.3529,  0.3517],
                [-1.5229,  0.5561,  0.2962]],
       
               [[ 1.6485,  1.3529,  0.3517],
                [-1.5229,  0.5561,  0.2962]],
       
               [[ 1.6485,  1.3529,  0.3517],
                [-1.5229,  0.5561,  0.2962]],
       
               [[ 1.6485,  1.3529,  0.3517],
                [-1.5229,  0.5561,  0.2962]],
       
               [[ 1.6485,  1.3529,  0.3517],
                [-1.5229,  0.5561,  0.2962]],
       
               [[ 1.6485,  1.3529,  0.3517],
                [-1.5229,  0.5561,  0.2962]]]),
    x: Batch(
           c: tensor([[[ 0.0233,  0.4830, -1.0290, -0.6119],
                       [-0.3239,  0.0862, -0.1516,  0.3273],
                       [-1.6574,  1.5067,  0.3015,  1.0849]],
         

In [26]:
%timeit Batch.stack(batches)

108 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.6485,  1.3529,  0.3517],
               [-1.5229,  0.5561,  0.2962],
               [ 1.6485,  1.3529,  0.3517],
               [-1.5229,  0.5561,  0.2962],
               [ 1.6485,  1.3529,  0.3517],
               [-1.5229,  0.5561,  0.2962],
               [ 1.6485,  1.3529,  0.3517],
               [-1.5229,  0.5561,  0.2962],
               [ 1.6485,  1.3529,  0.3517],
               [-1.5229,  0.5561,  0.2962],
               [ 1.6485,  1.3529,  0.3517],
               [-1.5229,  0.5561,  0.2962],
               [ 1.6485,  1.3529,  0.3517],
               [-1.5229,  0.5561,  0.2962],
               [ 1.6485,  1.3529,  0.3517],
               [-1.5229,  0.5561,  0.2962]]),
    x: Batch(
           c: tensor([[ 0.0233,  0.4830, -1.0290, -0.6119],
                      [-0.3239,  0.0862, -0.1516,  0.3273],
                      [-1.6574,  1.5067,  0.3015,  1.0849],
                      [ 0.0233,  0.4830, -1.0290, -0.6119],
                      [-0.3239,  

In [28]:
%timeit Batch.cat(batches)

197 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

600 µs ± 10 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
